# 📘 Model Predictive Control for AD (Production Ready)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/benmola/OpenAD-lib/blob/main/notebooks/05_MPC_Control_Full.ipynb)

**Optimal control** using AM2 model and do-mpc framework.

**⚠️ This notebook matches `examples/06_am2_mpc_control.py`**

---

## 📚 References
- **Data-Driven Control**: [Dekhici et al. (2024)](https://www.researchgate.net/publication/378298857)
- **do-mpc Framework**: [GitHub](https://github.com/do-mpc/do-mpc)

## 🔬 MPC Background

### What is Model Predictive Control?

**At each time step:**
1. **Predict** future behavior over horizon $N$
2. **Optimize** control actions to minimize objective
3. **Apply** only first control action
4. **Repeat** at next time step (receding horizon)

### Optimization Problem

$$\min_{u_0, ..., u_{N-1}} \sum_{k=0}^{N-1} L(x_k, u_k) + V_f(x_N)$$

Subject to:
$$x_{k+1} = f(x_k, u_k, d_k) \quad \text{(AM2 model)}$$
$$u_{min} \leq u_k \leq u_{max} \quad \text{(actuator limits)}$$
$$x_{min} \leq x_k \leq x_{max} \quad \text{(state constraints)}$$

Where:
- $x_k$ = state [S1, X1, S2, X2]
- $u_k$ = control (dilution rate D)
- $d_k$ = disturbances (S1_in, pH)
- $L$ = stage cost
- $V_f$ = terminal cost

### For Biogas Maximization

**Objective:**
$$\max Q = \max k_6 \mu_2(S_2) X_2$$

Equivalent to:
$$\min -Q$$

### Why MPC for AD?

✅ **Handles:**
- Constraints (D ∈ [0.01, 0.5] day⁻¹)
- Time-varying disturbances (inlet composition)
- Multi-objective optimization
- Process delays

✅ **Real-world applications:**
- Maximize biogas while maintaining stability
- Track VFA setpoint (prevent inhibition)
- Balance multiple performance metrics

## 1️⃣ Setup

In [ ]:
# Install with control dependencies (do-mpc, CasADi)
# !pip install git+https://github.com/benmola/OpenAD-lib.git

import sys
import os

IN_COLAB = 'google.colab' in sys.modules

if not IN_COLAB:
    sys.path.append(os.path.join(os.getcwd(), '..', 'src'))

print(f"Running in Colab: {IN_COLAB}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from openad_lib.models.mechanistic import AM2Parameters
from openad_lib.control import AM2MPC

print("✅ Imports successful!")

## 2️⃣ Control Scenario: Biogas Maximization

**Problem Statement:**
- **Goal:** Maximize biogas production
- **Manipulated variable:** Dilution rate D (day⁻¹)
- **Disturbance:** Inlet substrate concentration S1_in (varies sinusoidally)
- **Constraint:** 0.01 ≤ D ≤ 0.5 day⁻¹

**Challenge:**
- Too high D → washout (bacteria leave faster than they grow)
- Too low D → low throughput
- Must find optimal D considering future disturbances

In [ ]:
# Initialize with default calibrated parameters
print("Initializing AM2 parameters...")
params = AM2Parameters()

print(f"  µ1_max: {params.m1} day⁻¹")
print(f"  µ2_max: {params.m2} day⁻¹")
print(f"  Ki: {params.Ki} g COD/L")

In [ ]:
# Create MPC controller
print("\nInitializing MPC controller...")
controller = AM2MPC(params)

# MPC Tuning Parameters
sampling_time = 1.0  # 1 day (biological processes are slow)
horizon = 10         # 10-day prediction horizon
D_max = 0.5          # Maximum dilution rate

print(f"  Sampling time: {sampling_time} day")
print(f"  Prediction horizon: {horizon} days")
print(f"  Control horizon: {horizon} days")
print(f"  D bounds: [0.01, {D_max}] day⁻¹")

In [ ]:
# Setup controller (MATCH example exactly)
controller.setup_controller(
    sampling_time=sampling_time,
    horizon=horizon,
    objective_type='maximize_biogas',
    D_max=D_max
)

# Setup simulator for closed-loop
controller.setup_simulator(sampling_time=sampling_time)

print("✅ MPC controller configured")

## 3️⃣ Initial Conditions

Starting from a reasonable operating point:
- **S1** = 5.0 g COD/L (moderate substrate)
- **X1** = 1.0 g/L (acidogens)
- **S2** = 10.0 g COD/L (VFA at safe level)
- **X2** = 1.0 g/L (methanogens)

In [ ]:
# Initial state vector (MATCH example)
x0 = np.array([5.0, 1.0, 10.0, 1.0])  # [S1, X1, S2, X2]
controller.set_initial_state(x0)

print("Initial state:")
print(f"  S1 (substrate): {x0[0]:.2f} g COD/L")
print(f"  X1 (acidogens): {x0[1]:.2f} g/L")
print(f"  S2 (VFA): {x0[2]:.2f} g COD/L")
print(f"  X2 (methanogens): {x0[3]:.2f} g/L")

## 4️⃣ Closed-Loop Simulation

**Disturbance scenario:**
$$S_{1,in}(t) = 15 + 5 \sin\left(\frac{2\pi t}{20}\right)$$

**Simulation:**
- 50 days
- MPC adjusts D every day
- Observer feedback from simulated plant

In [ ]:
# Simulation parameters
n_days = 50
S1in_nominal = 15.0
pH_nominal = 7.0

# Storage
history = {
    'time': [],
    'D': [],
    'S1': [], 'X1': [], 'S2': [], 'X2': [],
    'Q': [],
    'S1in': []
}

print(f"🚀 Starting closed-loop simulation ({n_days} days)...\n")

for k in range(n_days):
    # Sinusoidal disturbance (MATCH example)
    S1in_k = S1in_nominal + 5.0 * np.sin(2 * np.pi * k / 20)
    pH_k = pH_nominal
    
    # MPC step
    u_opt, y_next = controller.run_step(S1in_val=S1in_k, pH_val=pH_k)
    
    # Extract states
    S1, X1, S2, X2 = float(y_next[0]), float(y_next[1]), float(y_next[2]), float(y_next[3])
    D = u_opt
    
    # Calculate biogas
    mu2 = (params.m2 * S2) / (params.K2 + S2 + (S2**2 / params.Ki))
    Q = params.c * params.k6 * mu2 * X2
    
    # Store
    history['time'].append(k)
    history['D'].append(D)
    history['S1'].append(S1)
    history['X1'].append(X1)
    history['S2'].append(S2)
    history['X2'].append(X2)
    history['Q'].append(Q)
    history['S1in'].append(S1in_k)
    
    if k % 10 == 0:
        print(f"Day {k:2d}: D={D:.4f}, Q={Q:.2f}, S2={S2:.2f}")

print("\n✅ Simulation complete!")
df = pd.DataFrame(history)

## 5️⃣ Results Analysis

**Key metrics:**
- Mean biogas production
- VFA stability (S2 should stay reasonable)
- D adaptation to disturbances

In [ ]:
print("📊 Performance Metrics:")
print("=" * 50)
print(f"  Mean biogas: {df['Q'].mean():.2f} ± {df['Q'].std():.2f}")
print(f"  Mean D: {df['D'].mean():.4f} day⁻¹")
print(f"  Mean S2 (VFA): {df['S2'].mean():.2f} g COD/L")
print(f"  Max S2: {df['S2'].max():.2f} g COD/L")
print(f"\n  D range: [{df['D'].min():.4f}, {df['D'].max():.4f}] day⁻¹")

In [ ]:
# Visualization (MATCH example layout)
plt.style.use('bmh')
fig, axes = plt.subplots(4, 1, figsize=(14, 12), sharex=True)

# Plot 1: Biogas Production
axes[0].plot(df['time'], df['Q'], linewidth=2, color='#27AE60')
axes[0].set_ylabel('Biogas (Q)', fontsize=12, fontweight='bold')
axes[0].set_title('MPC Control Results: Biogas Maximization', fontsize=14, pad=15)
axes[0].grid(True, alpha=0.3)

# Plot 2: Control Action (Dilution Rate)
axes[1].plot(df['time'], df['D'], linewidth=2, color='#E74C3C')
axes[1].axhline(y=D_max, color='red', linestyle='--', alpha=0.5, label=f'Max D={D_max}')
axes[1].set_ylabel('D (day⁻¹)', fontsize=12, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

# Plot 3: VFA (S2)
axes[2].plot(df['time'], df['S2'], linewidth=2, color='#F39C12')
axes[2].set_ylabel('S2 (VFA)', fontsize=12, fontweight='bold')
axes[2].grid(True, alpha=0.3)

# Plot 4: Disturbance
axes[3].plot(df['time'], df['S1in'], linewidth=2, color='#9B59B6', linestyle='--')
axes[3].set_ylabel('S1_in (Disturbance)', fontsize=12, fontweight='bold')
axes[3].set_xlabel('Time (days)', fontsize=12, fontweight='bold')
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 📝 Summary

This notebook demonstrated:

1. **MPC Setup** - AM2 model + do-mpc framework
2. **Biogas Maximization** - Objective formulation
3. **Disturbance Rejection** - Sinusoidal S1_in variation
4. **Closed-Loop Control** - 50-day simulation

### 🎯 Key Observations

**MPC Behavior:**
- D **increases** when S1_in is high (more substrate → higher throughput)
- D **decreases** when S1_in is low (prevent washout)
- Maintains **S2 stability** (VFA doesn't accumulate)

**Compare to open-loop constant D:**
- MPC adapts to disturbances
- Higher average biogas
- Better VFA stability

### 📚 MPC Tuning Guide

| Parameter | Typical Range | Effect |
|-----------|---------------|--------|
| **Horizon** | 5-20 days | Longer → better optimization, slower computation |
| **Sampling time** | 0.5-2 days | Faster → better disturbance rejection, more computation |
| **D bounds** | [0.01, 0.5] | Tighter → safer, less optimal |

### 🎓 Advanced Topics

**Try modifying:**
1. **Objective** - Track VFA setpoint instead of max biogas
2. **Constraints** - Add S2 < 15 g COD/L (prevent inhibition)
3. **Disturbance** - Step changes, random noise
4. **Model** - Use LSTM/MTGP instead of AM2

### Next Steps

- Compare with [AM2 open-loop](02_AM2_Modelling_Full.ipynb)
- Try [MTGP uncertainty propagation](04_MTGP_Prediction_Full.ipynb) in MPC
- Validate with [ADM1](01_ADM1_Tutorial_Full.ipynb) as "true" plant